# PyDentity Example - Researcher Side

In this notebook we'll be responding to a connection from a remote data Owner through our Agents. This notebook has the following phases:

<b>Begin at the Data Owner side</b>


6. Pull in dependencies
7. Instatiate the controller for our Agent
8. Set up a data message listener running as a service on our controller
9. Paste the invitation from the Data Owner notebook into the invitation variable here
10. Accept the invitation

<b>Carry on at the Data Owner side</b>
    
14. Send Trust Ping to the Data Owner

<b>Carry on at the Data Owner side</b>


You've established an end-end channel. Now begin Syft worker experimentation.

### 6. Pull in dependencies


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 7. Instatiate the controller for our Agent

In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

### 8. Set up a data message listener running as a service on our controller

In [3]:
def research_messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle research messages ", payload, connection_id)

research_message_listener = {
    "handler": research_messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([research_message_listener], defaults=True)

### 9. Paste the invitation from the Data Owner notebook into the invitation variable here;

In [4]:
#Paste in invitation from data_owner agent
invite = {'connection_id': 'b3431f7a-7a26-4425-88fa-742780a7b09d', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3cfc8191-010e-4982-8b17-5c9dd54e37d6', 'recipientKeys': ['Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG'], 'label': 'Bob', 'serviceEndpoint': 'http://192.168.65.3:8020'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiM2NmYzgxOTEtMDEwZS00OTgyLThiMTctNWM5ZGQ1NGUzN2Q2IiwgInJlY2lwaWVudEtleXMiOiBbIkR5NXVRR2Z4WlFYRzRRWEtjTW5Ka1c0YmlybTVXcUJKSjFMczZtbWU3QmtHIl0sICJsYWJlbCI6ICJCb2IiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly8xOTIuMTY4LjY1LjM6ODAyMCJ9'}

### 10. Accept the invitation, then move to Data Owner notebook

In [5]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
print(response)
# Print out accepted Invite and Researcher ID
print("Connection", response)
researcher_id = response["connection_id"]


ACCEPTED CONNECTION -  {'invitation_key': 'Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG', 'created_at': '2020-06-26 11:39:22.616439Z', 'accept': 'manual', 'their_label': 'Bob', 'connection_id': '9ce9f05c-0aff-4955-94ad-f0302afe146a', 'updated_at': '2020-06-26 11:39:22.629460Z', 'request_id': '8f6e61c6-bb5c-4741-a59d-c11c9c8eac91', 'invitation_mode': 'once', 'state': 'request', 'my_did': 'NAN12ecuQ2Kni6FoD9j3Gd', 'routing_state': 'none', 'initiator': 'external'}
{'invitation_key': 'Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG', 'created_at': '2020-06-26 11:39:22.616439Z', 'accept': 'manual', 'their_label': 'Bob', 'connection_id': '9ce9f05c-0aff-4955-94ad-f0302afe146a', 'updated_at': '2020-06-26 11:39:22.629460Z', 'request_id': '8f6e61c6-bb5c-4741-a59d-c11c9c8eac91', 'invitation_mode': 'once', 'state': 'request', 'my_did': 'NAN12ecuQ2Kni6FoD9j3Gd', 'routing_state': 'none', 'initiator': 'external'}
Connection {'invitation_key': 'Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG', 'created_at':

### 14. Send Trust Ping to the Data Owner

In [6]:
# Do a trust ping to ensure connection is accepted by data_owner
trust_ping = await agent_controller.connections.trust_ping(researcher_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA")
print(trust_ping)   

TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA
{'thread_id': '1a2fa9db-89df-4a3e-bb7e-427b9d1d625d'}


#### 14.1 Check if connection is active

In [7]:
# Print connection list
connection = await agent_controller.connections.get_connection(researcher_id)
print("RESEARCH AGENT CONNECTION")
print(connection)

Handle research messages  {'connection_id': '9ce9f05c-0aff-4955-94ad-f0302afe146a', 'message_id': 'ce356301-6b49-42cf-bc4a-ffae015d63d2', 'content': 'hello from data owner world! WOOPWOOP', 'state': 'received'} 9ce9f05c-0aff-4955-94ad-f0302afe146a
RESEARCH AGENT CONNECTION
{'invitation_key': 'Dy5uQGfxZQXG4QXKcMnJkW4birm5WqBJJ1Ls6mme7BkG', 'created_at': '2020-06-26 11:39:22.616439Z', 'accept': 'manual', 'their_label': 'Bob', 'connection_id': '9ce9f05c-0aff-4955-94ad-f0302afe146a', 'their_did': 'Mfc76nvc5xzWgGWoBjJcuL', 'updated_at': '2020-06-26 11:39:48.114142Z', 'request_id': '8f6e61c6-bb5c-4741-a59d-c11c9c8eac91', 'invitation_mode': 'once', 'state': 'active', 'my_did': 'NAN12ecuQ2Kni6FoD9j3Gd', 'routing_state': 'none', 'initiator': 'external'}


#### 14.2 Send a basic message over e2e channel

In [8]:
#send some basic messages
message = await agent_controller.connections.send_message(researcher_id,"hello from researcher world!")
print("BASIC MESSAGE - RESEARCH -> DATA")
print(message)

BASIC MESSAGE - RESEARCH -> DATA
{}


## Begin Syft Worker Experimentation

In [9]:
import torch
import syft as sy
from syft.serde import deserialize
from syft.serde import serialize
from syft.workers.base import BaseWorker


class AriesWorker(BaseWorker):
    def _send_msg(self, message: bin, location: BaseWorker) -> bin:
      return self._recv_msg(message)

    def _recv_msg(self, message: bin) -> bin:
      json_message = deserialize(message)
      print(json_message)
      result = self.recv_msg(message)
      result = deserialize(result)
      print(result)
      
      message = asyncio.get_event_loop().create_task(agent_controller.connections.send_message(researcher_id, str(result) ))
      print(message)
      return result

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/opt/conda/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [10]:
hook = sy.TorchHook(torch)
alice = AriesWorker(hook, id="alice")

In [11]:
def research_syft_handler(payload):
    connection_id = payload["connection_id"]
    alice._recv_msg(serialize(payload["content"]))
    print("Handle research messages ", payload, connection_id)

research_syft_listener = {
    "handler": research_syft_handler,
    "topic": "basicmessages"
}


In [12]:
agent_controller.register_listeners([research_syft_listener], defaults=False)